In [1]:
# Configura framework
from typing import Any
from sincpro_framework import UseFramework, Feature, DataTransferObject, ApplicationService, Middleware

framework = UseFramework("payment-system")


# Dependencies
class Database:
    """Database class to handle transactions"""

    def save_transaction(self, data):
        return f"saved_{id(data)}"

    def get_transaction(self, tx_id):
        return {"id": tx_id, "status": "completed"}


class VisaAdapter:
    """Adapter for Visa payment processing"""

    def make_request(self, card_data):
        return {"approved": True, "tx_id": "visa_123"}

    def check_status(self, tx_id):
        return "approved"

def check_health():
    """Health check function for the framework"""
    return {"status": "ok", "version": "1.0.0"}

# Middleware
class ValidatePayment(Middleware):
    """Middleware to validate payment data"""

    def __call__(self, dto: Any) -> Any:
        """Validate payment and raise error if invalid"""
        if hasattr(dto, 'amount') and dto.amount <= 0:
            raise ValueError("Amount must be positive")
        if hasattr(dto, 'user_id') and not dto.user_id.strip():
            raise ValueError("User ID is required")
        return dto


database = Database()
visa_adapter = VisaAdapter()

framework.add_dependency("database", database)
framework.add_dependency("visa_adapter", visa_adapter)
framework.add_dependency("health_check", check_health)
framework.add_middleware(ValidatePayment())


class Feat(Feature):
    database: Database
    visa_adapter: VisaAdapter


class Services(ApplicationService):
    database: Database
    visa_adapter: VisaAdapter



read yaml file /home/andru1236/dev/sincpro_framework/sincpro_framework/conf/sincpro_framework_conf.yml for config DefaultFrameworkConfig


In [2]:
class PaymentCommand(DataTransferObject):
    """PaymentCommand"""
    card_number: str
    amount: float
    merchant_id: str


class PaymentResponse(DataTransferObject):
    """PaymentResponse"""
    transaction_id: str
    status: str
    amount: float


class RefundCommand(DataTransferObject):
    """RefundCommand"""

    transaction_id: str
    amount: float
    reason: str


class RefundResponse(DataTransferObject):
    """RefundResponse"""
    refund_id: str
    status: str
    amount: float


class ValidateCommand(DataTransferObject):
    """ValidateCommand"""
    card_number: str
    cvv: str


class ValidateResponse(DataTransferObject):
    """ValidateResponse"""
    is_valid: bool
    card_type: str

In [3]:
@framework.feature(PaymentCommand)
class PaymentFeature(Feat):
    """Feature to handle payment processing"""

    def execute(self, dto: PaymentCommand) -> PaymentResponse:
        """Execute payment processing
        Args:
            dto (PaymentCommand): Data transfer object containing payment details
        Returns:
            PaymentResponse: Response containing transaction details
        """
        visa_result = self.visa_adapter.make_request({"card": dto.card_number})
        tx_id = self.database.save_transaction(dto)
        return PaymentResponse(
            transaction_id=tx_id,
            status="success",
            amount=dto.amount
        )


@framework.feature(RefundCommand)
class RefundFeature(Feat):
    """Feature to handle refund processing"""

    def execute(self, dto: RefundCommand) -> RefundResponse:
        """Execute refund processing
        Args:
            dto (RefundCommand): Data transfer object containing refund details
        Returns:
            RefundResponse: Response containing refund details
        """
        # Assuming we have a method to get the original transaction
        # and a method to process the refund through the Visa adapter
        original_tx = self.database.get_transaction(dto.transaction_id)
        refund_result = self.visa_adapter.make_request({"refund": dto.amount})
        refund_id = self.database.save_transaction(dto)
        return RefundResponse(
            refund_id=refund_id,
            status="refunded",
            amount=dto.amount
        )


@framework.feature(ValidateCommand)
class ValidateFeature(Feat):
    """Feature to validate payment card details"""

    def execute(self, dto: ValidateCommand) -> ValidateResponse:
        """Execute card validation
        Args:
            dto (ValidateCommand): Data transfer object containing card details
        Returns:
            ValidateResponse: Response indicating if the card is valid
        """
        # Assuming we have a method to check the card status through the Visa adapter
        # and return whether it is approved or not
        validation = self.visa_adapter.check_status(dto.card_number)
        return ValidateResponse(
            is_valid=validation == "approved",
            card_type="visa"
        )




2025-07-30T04:50:37.828341Z [debug    ] Registering feature: [PaymentCommand] app_name=payment-system filename=ioc.py func_name=_register_service lineno=104
2025-07-30T04:50:37.828957Z [debug    ] Registering feature: [RefundCommand] app_name=payment-system filename=ioc.py func_name=_register_service lineno=104
2025-07-30T04:50:37.829297Z [debug    ] Registering feature: [ValidateCommand] app_name=payment-system filename=ioc.py func_name=_register_service lineno=104


In [4]:
class MakeTransactionCommand(DataTransferObject):
    """MakeTransactionCommand"""
    card_number: str
    amount: float
    merchant_id: str


class MakeTransactionResponse(DataTransferObject):
    """MakeTransactionResponse"""
    transaction_id: str
    status: str
    amount: float


@framework.app_service(MakeTransactionCommand)
class MakeTransaction(Services):
    def execute(self, dto: MakeTransactionCommand) -> MakeTransactionResponse:
        """Execute transaction processing
        Args:
            dto (MakeTransactionCommand): Data transfer object containing transaction details
        Returns:
            MakeTransactionResponse: Response containing transaction details
        """
        # Validate the payment
        validation = self.feature_bus.execute(ValidateCommand(card_number=dto.card_number, cvv="123"))

        if not validation.is_valid:
            raise ValueError("Invalid card details")

        # Process the payment
        payment = self.feature_bus.execute(
            PaymentCommand(card_number=dto.card_number, amount=dto.amount, merchant_id=dto.merchant_id)
        )

        return MakeTransactionResponse(
            transaction_id=payment.transaction_id,
            status=payment.status,
            amount=payment.amount
        )

2025-07-30T04:50:38.892578Z [debug    ] Registering application service: [MakeTransactionCommand] app_name=payment-system filename=ioc.py func_name=_register_service lineno=104


In [5]:
response = framework(MakeTransactionCommand(
    card_number="4111111111111111",
    amount=100.0,
    merchant_id="merchant_123"
), MakeTransactionResponse)
response

2025-07-30T04:50:40.217771Z [debug    ] Framework bus created          app_name=payment-system filename=bus.py func_name=__init__ lineno=137
2025-07-30T04:50:40.218243Z [debug    ] Registered features: {'ValidateCommand', 'RefundCommand', 'PaymentCommand'} app_name=payment-system filename=bus.py func_name=__init__ lineno=138
2025-07-30T04:50:40.218496Z [debug    ] Registered app services: {'MakeTransactionCommand'} app_name=payment-system filename=bus.py func_name=__init__ lineno=141
2025-07-30T04:50:40.218753Z [info     ] Executing app service dto: [MakeTransactionCommand] app_name=payment-system filename=bus.py func_name=execute lineno=93
2025-07-30T04:50:40.218972Z [debug    ] MakeTransactionCommand(card_number='4111111111111111' amount=100.0 merchant_id='merchant_123') app_name=payment-system filename=bus.py func_name=execute lineno=96
2025-07-30T04:50:40.219169Z [info     ] Executing feature dto: [ValidateCommand] app_name=payment-system filename=bus.py func_name=execute lineno=42

MakeTransactionResponse(transaction_id='saved_140052569725824', status='success', amount=100.0)

In [6]:
from sincpro_framework.generate_documentation import build_documentation


In [7]:
build_documentation(framework)

✅ Generated: generated_docs/index.md
✅ Generated: generated_docs/dependencies.md
✅ Generated: generated_docs/middlewares.md
✅ Generated: generated_docs/application-services.md
✅ Generated: generated_docs/features.md
✅ Generated: generated_docs/dtos.md
✅ Generated: generated_docs/mkdocs_nav.yml
✅ All documentation files saved to: generated_docs/


/home/andru1236/dev/sincpro_framework/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "mro" in "ClassMetadata" shadows an attribute in parent "BaseModel"
  warnings.warn(
